# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
cities = pd.read_csv("../output_data/weatherpy_data.csv", encoding="utf-8")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,toora-khem,99,RU,1603251245,77,52.47,96.11,34.81,2.98
1,1,airai,13,TL,1603251245,35,-8.93,125.41,81.61,4.59
2,2,manta,75,EC,1603251245,78,-0.95,-80.73,73.40,12.75
3,3,sorland,40,NO,1603251174,69,67.67,12.69,37.40,6.93
4,4,markova,96,UA,1603251094,76,48.68,24.41,38.50,6.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
humidity = cities["Humidity"].astype(float)
max_humidity = humidity.max()

print(max_humidity)

city_locations = cities[["Lat", "Lng"]]

100.0


In [23]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)


heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity,
                                 max_intensity=max_humidity, 
                                 dissipating=False, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
clean_city_df = cities.loc[(cities["Max Temp"] > 70) &
                                          (cities["Max Temp"] < 80) &
                                          (cities["Wind Speed"] < 10) &
                                          (cities["Cloudiness"] == 0)
                                          , :]
clean_city_df = clean_city_df.dropna(how='any')
clean_city_df.reset_index(inplace=True)
del clean_city_df['index']

In [27]:
clean_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,46,savelugu,0,GH,1603251253,83,9.62,-0.83,76.01,4.12
1,110,angoche,0,MZ,1603251261,80,-16.23,39.91,75.45,5.12
2,118,san cristobal,0,ML,1603251132,54,13.30,-4.90,78.66,5.79
3,176,sao joao da barra,0,IN,1603251268,71,20.51,76.21,79.72,3.44
4,196,petauke,0,ZM,1603251270,48,-14.24,31.33,71.73,6.69
5,212,tessalit,0,ML,1603251273,18,20.20,1.01,77.65,8.77
6,235,garowe,0,SO,1603251278,72,8.41,48.48,77.70,6.76
7,265,san carlos de bariloche,0,ML,1603251003,54,13.30,-4.90,78.66,5.79
8,306,san patricio,0,ML,1603251003,54,13.30,-4.90,78.66,5.79
9,315,san quintin,0,ML,1603251003,54,13.30,-4.90,78.66,5.79


In [30]:
hotel_df = clean_city_dfcity_df_city_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))